# Item-Based Collaborative Filtering - Cosine Similarity

## 1. Set-up
import dependent packages and declare consts

In [2]:
# package initialization

import pandas as pd
import numpy as np
from surprise import BaselineOnly, KNNBasic
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from sklearn import metrics

In [28]:
# consts

# DATASET = '../../data/extracted/LON-A/London_Attractions_Complete_Review.csv'
DATASET = '../../data/extracted/NYC-R/New_York_City_Restaurant_Complete_Review.csv'
OCCURENCE_THRESHOLD = 5

## 2. Read Dataset
read dataset in csv format into pandas.DataFrame

In [29]:
# read dataset

df = pd.read_csv(DATASET, sep='\t')

In [30]:
# print dataset information

print("Columns: \n", list(df.columns))
print("\nShape: \n", df.shape)

Columns: 
 ['Unnamed: 0', 'Unnamed: 0.1', 'rtime', 'rquote', 'iid', 'rrate', 'rid', 'uage', 'ucity', 'ucountry', 'ugender', 'uhometown', 'uid_index', 'ulevel', 'uname_y', 'usince', 'ustate', 'ustyle', 'iattribute', 'ilocality', 'iname', 'ipopularity', 'ipost', 'iprice', 'irating', 'iregion', 'istreet', 'itag']

Shape: 
 (129964, 28)


## 3. Data Preprocessing

* Retain users/items with at least five ratings only
* Data splitting
  - the latest 20% interactions (by time)
  - Randomly split the remaining data into training (70%) and validation (10%) sets
* Transform the ratings into binary implicit feedback as ground truth, indicating whether the user has interacted with the specific item

In [6]:
def sort_by_time(df):
    
    # here we use 'rid' for sorting becaz it's auto incrementing
    return df.sort_values(by=['rid'], ascending=True)

In [7]:
def filter_by_occurrence(df, column, threshold):
    return df.groupby(column).filter(lambda x: len(x) >= threshold)

In [8]:
def convert_binary(df):
    df.loc[df['rrate'] != "None", 'rrate'] = 1.0
    df.loc[df['rrate'] == "None", 'rrate'] = 0.0
    return df

In [9]:
def data_preprocess(dataframe):
    
    # sort by time (ascending order)
    df = sort_by_time(dataframe)
    
    # retrieve needed columns
    df = df[['uid_index', 'iid', 'rrate']]
    
    # convert ratings into binarys
    df = convert_binary(df)
    
    df['rrate'] = pd.to_numeric(df['rrate'])
    
    # Retain users/items with at least five ratings only
    df = filter_by_occurrence(df, 'iid', OCCURENCE_THRESHOLD)
    df = filter_by_occurrence(df, 'uid_index', OCCURENCE_THRESHOLD)
    
    # split dataset into training set, validation set and test set
    users = df.groupby('uid_index')
    
    test_df = pd.DataFrame()
    train_validation_df = pd.DataFrame()
    
    # for each user, get its latest 20% rating as test set
    for uid in users.size().to_dict().keys():
        user = users.get_group(uid)
        split_idx = int(len(user)*0.8)
        test_df = test_df.append(user.iloc[split_idx:])
        train_validation_df = train_validation_df.append(user.iloc[:split_idx])
    
    train_validation_df = train_validation_df.reindex(np.random.permutation(train_validation_df.index)) # shuffle
    train_df = train_validation_df.iloc[:int(len(train_validation_df)*0.875)]
    validation_df = train_validation_df.iloc[int(len(train_validation_df)*0.875):]
    
    return (train_df, validation_df, test_df)

In [31]:
# dataset preprocessing

train_df, validation_df, test_df = data_preprocess(df)
print("training set size: ", train_df.shape)
print("validation set size: ", validation_df.shape)
print("test set size: ", test_df.shape)

training set size:  (80722, 3)
validation set size:  (11532, 3)
test set size:  (29517, 3)


## 4. Load into Surprise

In [32]:
reader = Reader(rating_scale=(0, 1))
train_dataset = Dataset.load_from_df(train_df, reader).build_full_trainset()

## 5. Model declaration & Fitting

Collaborative Filtering - KNN:

\begin{equation}
\hat{r}_{ui} = \frac{
\sum\limits_{j \in N^k_u(i)} \text{sim}(i, j) \cdot r_{uj}}
{\sum\limits_{j \in N^k_u(i)} \text{sim}(i, j)}
\end{equation}

**Parameters**
- `k` – The max number of neighbors to take into account for aggregation, defaults to `40`

In [12]:
# compute consine similarity between items
sim_options = {'name': 'cosine', 'user_based': False}

K_MAX = 40

In [13]:
algorithm = KNNBasic(k=K_MAX, sim_options=sim_options)

In [14]:
algorithm.fit(train_dataset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


E:\ProgramData\Anaconda3\envs\surprise\lib\site-packages\surprise\prediction_algorithms\algo_base.py:248: RuntimeWarning: invalid value encountered in double_scalars
  sim = construction_func[name](*args)


## 6. Prediction

In [15]:
def predict(algorithm, dataframe):
    z = []
    y = []
    for i in range(len(dataframe)):
        user = dataframe.iloc[i][0]
        item = dataframe.iloc[i][1]
        rating = dataframe.iloc[i][2]
        prediction = algorithm.predict(user, item, r_ui=rating, verbose=False)
        
        if prediction.details['was_impossible'] == False:
            z.append(prediction.est)
            y.append(rating)
    return (np.array(z, dtype=np.float32), np.array(y, dtype=np.int))

In [16]:
validation_z, validation_y = predict(algorithm, validation_df)

In [17]:
test_z, test_y = predict(algorithm, test_df)

## 7. Evaluation

AUC metric

In [18]:
def evaluate_auc(z, y):
    return metrics.roc_auc_score(y, z)

In [19]:
print("validation AUC: ", evaluate_auc(validation_z, validation_y))
print("test AUC: ", evaluate_auc(test_z, test_y))

validation AUC:  0.44881078535264324
test AUC:  0.4852904654386654


LogLoss metric

In [20]:
# assume parameters z & y are ndarray
def evaluate_logloss(z, y):
    zz = np.ones((z.shape[0], 2))
    zz[:, 0] -= z
    zz[:, 1] = z
    return metrics.log_loss(y, zz)

In [21]:
print("validation LogLoss: ", evaluate_logloss(validation_z, validation_y))
print("test LogLoss: ", evaluate_logloss(test_z, test_y))

validation LogLoss:  2.6252315718365558
test LogLoss:  2.1989300960122096


NDCG metric

In [22]:
# assume parameters z & y are ndarray
def evaluate_ndcg(z, y):
    return metrics.ndcg_score(np.expand_dims(y, axis=0), np.expand_dims(z, axis=0), k=5)

In [23]:
print("validation NDCG@5: ", evaluate_ndcg(validation_z, validation_y))
print("test NDCG@5: ", evaluate_ndcg(test_z, test_y))

validation NDCG@5:  0.9061900191938579
test NDCG@5:  0.9198132637649319


## Experiments

In [34]:
def train_ks(start=3, end=20, step=1):

    history = []
    for k in range(start, end+step, step):
        
        print("Using K =", k)
        sim_options = {'name': 'cosine', 'user_based': False}
        algorithm = KNNBasic(k=k, sim_options=sim_options)
        
        algorithm.fit(train_dataset)
        validation_z, validation_y = predict(algorithm, validation_df)
        test_z, test_y = predict(algorithm, test_df)
        
        history.append({
            'k': k,
            'val_auc': evaluate_auc(validation_z, validation_y),
            'test_auc': evaluate_auc(test_z, test_y),
            'val_logloss': evaluate_logloss(validation_z, validation_y),
            'test_logloss': evaluate_logloss(test_z, test_y),
            'val_ndcg': evaluate_ndcg(validation_z, validation_y),
            'test_ndcg': evaluate_ndcg(test_z, test_y)
        })
    return history

In [41]:
history = train_ks(start=21, end=30, step=1)

Using K = 21
Computing the cosine similarity matrix...


E:\ProgramData\Anaconda3\envs\surprise\lib\site-packages\surprise\prediction_algorithms\algo_base.py:248: RuntimeWarning: invalid value encountered in double_scalars
  sim = construction_func[name](*args)


Done computing similarity matrix.
Using K = 22
Computing the cosine similarity matrix...


E:\ProgramData\Anaconda3\envs\surprise\lib\site-packages\surprise\prediction_algorithms\algo_base.py:248: RuntimeWarning: invalid value encountered in double_scalars
  sim = construction_func[name](*args)


Done computing similarity matrix.
Using K = 23
Computing the cosine similarity matrix...


...

Done computing similarity matrix.
Using K = 29
Computing the cosine similarity matrix...


E:\ProgramData\Anaconda3\envs\surprise\lib\site-packages\surprise\prediction_algorithms\algo_base.py:248: RuntimeWarning: invalid value encountered in double_scalars
  sim = construction_func[name](*args)


Done computing similarity matrix.
Using K = 30
Computing the cosine similarity matrix...


E:\ProgramData\Anaconda3\envs\surprise\lib\site-packages\surprise\prediction_algorithms\algo_base.py:248: RuntimeWarning: invalid value encountered in double_scalars
  sim = construction_func[name](*args)


Done computing similarity matrix.


In [42]:
print("| K | validation AUC | validation LogLoss | validation NDCG@5 | testing AUC | testing LogLoss | testing NDCG@5 |")
print("|:-- | -- | -- | -- | -- | -- | -- |")
for his in history:
    print("| k={} | {:.5f} | {:.5f} | {:.5f} | {:.5f} | {:.5f} | {:.5f} |".format(
        his['k'],
        his['val_auc'],
        his['val_logloss'],
        his['val_ndcg'],
        his['test_auc'],
        his['test_logloss'],
        his['test_ndcg'],
    ))

| K | validation AUC | validation LogLoss | validation NDCG@5 | testing AUC | testing LogLoss | testing NDCG@5 |
|:-- | -- | -- | -- | -- | -- | -- |
| k=21 | 0.95310 | 0.17524 | 0.99751 | 0.97260 | 0.10368 | 0.99836 |
| k=22 | 0.95310 | 0.17524 | 0.99751 | 0.97260 | 0.10368 | 0.99836 |
| k=23 | 0.95342 | 0.17521 | 0.99751 | 0.97266 | 0.10367 | 0.99836 |
| k=24 | 0.95342 | 0.17518 | 0.99751 | 0.97273 | 0.10366 | 0.99836 |
| k=25 | 0.95373 | 0.17515 | 0.99751 | 0.97292 | 0.10363 | 0.99837 |
| k=26 | 0.95372 | 0.17515 | 0.99751 | 0.97292 | 0.10364 | 0.99836 |
| k=27 | 0.95372 | 0.17515 | 0.99751 | 0.97292 | 0.10364 | 0.99836 |
| k=28 | 0.95372 | 0.17515 | 0.99751 | 0.97310 | 0.10361 | 0.99836 |
| k=29 | 0.95372 | 0.17515 | 0.99751 | 0.97323 | 0.10360 | 0.99836 |
| k=30 | 0.95372 | 0.17515 | 0.99751 | 0.97329 | 0.10359 | 0.99836 |


## Experiment Results

LON-A dataset:  

| settings | validation AUC | validation LogLoss | validation NDCG@5 | testing AUC | testing LogLoss | testing NDCG@5 |
|:-- | -- | -- | -- | -- | -- | -- |
| **k=3*** | 0.49650 | 2.55242 | 0.92808 | 0.52433 | 2.12324 | 0.93968 |
| k=4 | 0.48705 | 2.49657 | 0.92450 | 0.51455 | 2.13440 | 0.93399 |
| k=5 | 0.47963 | 2.53492 | 0.92124 | 0.50672 | 2.17230 | 0.93057 |
| k=6 | 0.46985 | 2.59390 | 0.91747 | 0.50322 | 2.18739 | 0.92855 |
| k=7 | 0.46930 | 2.58139 | 0.91665 | 0.49993 | 2.20220 | 0.92678 |
| k=8 | 0.46727 | 2.59588 | 0.91511 | 0.49648 | 2.19766 | 0.92571 |
| k=9 | 0.46239 | 2.61721 | 0.91331 | 0.49478 | 2.19797 | 0.92488 |
| k=10 | 0.46077 | 2.62906 | 0.91217 | 0.49323 | 2.19956 | 0.92423 |

NYC-R dataset:  

| settings | validation AUC | validation LogLoss | validation NDCG@5 | testing AUC | testing LogLoss | testing NDCG@5 |
|:-- | -- | -- | -- | -- | -- | -- |
| **k=3*** | 0.91139 | 0.16775 | 0.99766 | 0.95093 | 0.10484 | 0.99840 |
| k=4 | 0.92084 | 0.17893 | 0.99748 | 0.95788 | 0.10601 | 0.99835 |
| k=5 | 0.93513 | 0.17126 | 0.99769 | 0.96241 | 0.10535 | 0.99835 |
| k=6 | 0.93385 | 0.17764 | 0.99750 | 0.96576 | 0.10466 | 0.99836 |
| k=7 | 0.93859 | 0.17677 | 0.99751 | 0.96624 | 0.10684 | 0.99829 |
| k=8 | 0.94221 | 0.17635 | 0.99751 | 0.96707 | 0.10665 | 0.99829 |
| k=9 | 0.94542 | 0.17604 | 0.99751 | 0.96819 | 0.10649 | 0.99829 |
| k=10 | 0.94623 | 0.17604 | 0.99751 | 0.96898 | 0.10650 | 0.99829 |
| k=11 | 0.94784 | 0.17584 | 0.99751 | 0.96953 | 0.10640 | 0.99829 |
| k=12 | 0.94862 | 0.17569 | 0.99751 | 0.97013 | 0.10631 | 0.99829 |
| k=13 | 0.94973 | 0.17557 | 0.99751 | 0.97027 | 0.10628 | 0.99829 |
| k=14 | 0.94973 | 0.17556 | 0.99751 | 0.97146 | 0.10383 | 0.99836 |
| k=15 | 0.95043 | 0.17552 | 0.99751 | 0.97160 | 0.10380 | 0.99836 |
| k=16 | 0.95043 | 0.17554 | 0.99751 | 0.97167 | 0.10379 | 0.99836 |
| k=17 | 0.95112 | 0.17546 | 0.99751 | 0.97201 | 0.10375 | 0.99836 |
| k=18 | 0.95180 | 0.17541 | 0.99751 | 0.97234 | 0.10371 | 0.99836 |
| k=19 | 0.95213 | 0.17537 | 0.99751 | 0.97253 | 0.10369 | 0.99836 |
| k=20 | 0.95278 | 0.17531 | 0.99751 | 0.97253 | 0.10369 | 0.99836 |
| k=21 | 0.95310 | 0.17524 | 0.99751 | 0.97260 | 0.10368 | 0.99836 |
| k=22 | 0.95310 | 0.17524 | 0.99751 | 0.97260 | 0.10368 | 0.99836 |
| k=23 | 0.95342 | 0.17521 | 0.99751 | 0.97266 | 0.10367 | 0.99836 |
| k=24 | 0.95342 | 0.17518 | 0.99751 | 0.97273 | 0.10366 | 0.99836 |
| **k=25*** | 0.95373 | 0.17515 | 0.99751 | 0.97292 | 0.10363 | 0.99837 |
| k=26 | 0.95372 | 0.17515 | 0.99751 | 0.97292 | 0.10364 | 0.99836 |
| k=27 | 0.95372 | 0.17515 | 0.99751 | 0.97292 | 0.10364 | 0.99836 |
| k=28 | 0.95372 | 0.17515 | 0.99751 | 0.97310 | 0.10361 | 0.99836 |
| k=29 | 0.95372 | 0.17515 | 0.99751 | 0.97323 | 0.10360 | 0.99836 |
| k=30 | 0.95372 | 0.17515 | 0.99751 | 0.97329 | 0.10359 | 0.99836 |